In [37]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import requests
import pandas as pd

In [38]:
with open('DevBgLinks.txt') as file:
    urls = [url.rstrip() for url in file]

urls

['https://dev.bg/company/jobs/back-end-development/',
 'https://dev.bg/company/jobs/operations/',
 'https://dev.bg/company/jobs/mobile-development/',
 'https://dev.bg/company/jobs/hardware-and-engineering/',
 'https://dev.bg/company/jobs/front-end-development/',
 'https://dev.bg/company/jobs/quality-assurance/',
 'https://dev.bg/company/jobs/data-science/',
 'https://dev.bg/company/jobs/customer-support/',
 'https://dev.bg/company/jobs/full-stack-development/',
 'https://dev.bg/company/jobs/pm-ba-and-more/',
 'https://dev.bg/company/jobs/erp-crm-development/',
 'https://dev.bg/company/jobs/ui-ux-and-arts/',
 'https://dev.bg/company/jobs/technical-support/']

In [39]:
class Job_offer:
  def __init__(self, company_name, job_title,location,date_posted):
    self.company_name = company_name
    self.job_title = job_title
    self.location = location
    self.date_posted = date_posted
    self.techstack = []

In [50]:
def get_soup(page_number,url):
    url = f'{url}?_paged={page_number}'

    driver = webdriver.Chrome()
    try:
        driver.get(url)

        time.sleep(1)
    
        page_source = driver.page_source
        soup = bs(page_source, 'html.parser')
    
    finally:
        driver.quit()
        return soup

In [51]:
def extract_job_object(job):
    company = job.find('span', class_ = 'company-name').text.strip().replace('\n','')
    job_title = job.find('h6', class_ = 'job-title').text.strip().replace('\n','')
    date = job.find('span', class_ = 'date').text.strip().replace('\n','')
    location = job.find('span', class_ = 'badge').text.strip().replace('\n','').replace('Комбиниран модел на работа, позволяващ на служителите съчетание между работа в офиса и работа от разстояние.','').replace('            ','/')
    
    tech_stack = set([])
    tech_stack_images = job.find_all('img', class_='attachment-medium size-medium')
    
    for image in tech_stack_images:
        tech_stack.add(image.get('title'))
    
    job_offer = Job_offer(company,job_title,location,date)
    job_offer.tech_stack = tech_stack

    return job_offer

In [58]:
job_offers = []
for url in urls:
    soup = get_soup(1,url) # get first page so we can acces pages count
    max_pages_text = soup.find('a', class_= 'facetwp-page last').text
    max_pages = int(max_pages_text)
    for page_number in range(1,max_pages+1):
        soup = get_soup(page_number,url)
        jobs_in_page = soup.find_all('div', class_ = 'job-list-item')
        for job in jobs_in_page:
            try:
                job_offer = extract_job_object(job)
                job_offers.append(job_offer)
            except:
                print(f"An exception :{job.find('h6',class_='job-title ab-title-placeholder ab-cb-title-placeholder').text}") 

An exception : Service Manager 
An exception : SAP TRM LEAD DEVELOPER/ARCHITECT 
An exception : SAP CONSULTANT FOR SUPPLIER RISKS SOLUTIONS 


In [53]:
print(len(job_offers))

2559


In [60]:
table_headers = ['company_name','job_title','location','date_posted','tech_stack']

df = pd.DataFrame(columns = table_headers)
df

,company_name,job_title,location,date_posted,tech_stack


In [61]:
for offer in job_offers:
    length = len(df)
    row_data = [offer.company_name,offer.job_title,offer.location,offer.date_posted,', '.join(offer.tech_stack)] 
    df.loc[length] = row_data

In [62]:
df.to_csv('DevBG_Data.csv',index=False)